In [2]:
from hazm import *
import hazm
import math
import plotly.express as px
import plotly.graph_objects as go
import time
from cleantext import clean
from tqdm import tqdm


In [3]:
import pandas as pd
import numpy as np
import fileinput

In [4]:
import argparse
import sys
import re

In [5]:
!pip3 install requests
import emoji
import string
import json
import config as conf
import requests
from elasticsearch import Elasticsearch

/home/mohi/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/mohi/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [6]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

In [7]:
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [8]:

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

[nltk_data] Downloading package stopwords to /home/mohi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df = pd.read_csv('CovidTweet.csv')
df = df["clean_tweet"].head(10000)

In [11]:
def sent_to_words(sentences):
    for sent in sentences:
        
        
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = df.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['call', 'leader', 'help', 'protect', 'refuge', 'covid', 'provid', 'qualiti', 'health', 'care']]


In [26]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!

In [28]:
LDA = gensim.models.ldamodel.LdaModel
dict_ = corpora.Dictionary(data_words)
doc_term_matrix=[dict_.doc2bow(i) for i in data_ready]
coherence = []
for k in range(2,6):
    ldaModel = LDA(doc_term_matrix, num_topics=k, id2word=dict_,chunksize=1000, passes=1, random_state=0, update_every=1, eval_every=None)
        
    cm = gensim.models.coherencemodel.CoherenceModel(model=ldaModel, texts=data_ready, dictionary=dict_,
     coherence='c_v')
    coherence.append((k,cm.get_coherence()))
x_val = [x[0] for x in coherence]
y_val = [x[1] for x in coherence]
maxCoh = max(y_val)

NumOfTopics = 0
for i in range(4):
    if y_val[i] == maxCoh:
        NumOfTopics = i+1

print(NumOfTopics)



4


In [29]:
ldaModel = LDA(doc_term_matrix, num_topics=NumOfTopics, id2word=dict_,chunksize=1000, passes=1, random_state=0, update_every=1, eval_every=None)

In [30]:
ldaModel.print_topics()

[(0,
  '0.072*"covid" + 0.034*"test" + 0.014*"die" + 0.013*"posit" + 0.012*"coronaviru" + 0.011*"stay" + 0.010*"new" + 0.009*"fight" + 0.009*"worker" + 0.008*"world"'),
 (1,
  '0.016*"covid" + 0.014*"state" + 0.014*"doctor" + 0.013*"govern" + 0.012*"week" + 0.011*"trump" + 0.010*"find" + 0.008*"give" + 0.007*"support" + 0.007*"plea"'),
 (2,
  '0.055*"covid" + 0.036*"death" + 0.022*"case" + 0.014*"today" + 0.012*"due" + 0.012*"new" + 0.010*"patient" + 0.010*"report" + 0.009*"prosper" + 0.008*"follow"'),
 (3,
  '0.054*"disinfect" + 0.038*"covid" + 0.019*"ingest" + 0.015*"trump" + 0.013*"inject" + 0.012*"presid" + 0.012*"look" + 0.010*"work" + 0.010*"suggest" + 0.008*"first"')]

In [31]:
count = 0
for i in ldaModel[doc_term_matrix]:
    print('doc :', count, i)
    count+=1

doc : 0 [(0, 0.035113826), (1, 0.9012806), (2, 0.031784628), (3, 0.031820916)]
doc : 1 [(0, 0.051416386), (1, 0.84730613), (2, 0.050729122), (3, 0.05054833)]
doc : 2 [(0, 0.029018737), (1, 0.91427165), (2, 0.028566398), (3, 0.0281432)]
doc : 3 [(0, 0.3576273), (1, 0.050249368), (2, 0.54156274), (3, 0.050560582)]
doc : 4 [(0, 0.0847451), (1, 0.084480405), (2, 0.08678321), (3, 0.7439913)]
doc : 5 [(0, 0.84835935), (1, 0.05005739), (2, 0.050175104), (3, 0.05140812)]
doc : 6 [(0, 0.60699886), (1, 0.12727521), (2, 0.13479482), (3, 0.13093111)]
doc : 7 [(0, 0.03023281), (1, 0.80449474), (2, 0.13678803), (3, 0.028484372)]
doc : 8 [(0, 0.13115954), (1, 0.1250486), (2, 0.6184513), (3, 0.1253406)]
doc : 9 [(0, 0.033807285), (1, 0.8392685), (2, 0.032530844), (3, 0.094393365)]
doc : 10 [(0, 0.0841853), (1, 0.08336191), (2, 0.7475415), (3, 0.084911264)]
doc : 11 [(0, 0.035841677), (1, 0.8920929), (2, 0.035946246), (3, 0.03611916)]
doc : 12 [(0, 0.042404987), (1, 0.041835766), (2, 0.87371296), (3, 0